In [1]:
import collections
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from venn import venn
import numpy as np
import math
import scipy

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_37298/2185171821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
clinical_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/figurecode/data/20221109_TRACERx421_all_patient_df.rds')
clinical_df = pd.DataFrame(clinical_df[None])
clinical_df.head()

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,0,1849,0,1849,NaN,No rec,True
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,1,1362,1,1362,0,1362,NaN,No rec,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,2224,1,2224,1,1935,0,2224,Second primary lung,No rec,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,2365,1,1143,1,1143,1,1143,Intrathoracic,Intrathoracic,True
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,986,1,917,1,917,1,917,Intrathoracic,Intrathoracic,True


In [3]:
tumor_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_all_tumour_df.rds')
tumor_df = pd.DataFrame(tumor_df[None])
tumor_df.head()

,tumour_id,patient_id,clinical_sex,age,ethnicity,Histology_per_tumour_id_muttable,histology_3,LUAD_pred_subtype_with.IMA_per_tumour,site_per_lesion,pT_stage_per_lesion,pN_stage_per_lesion,size_pathology_per_lesion,vascular_invasion_per_lesion,pleural_invasion_per_lesion,smoking_status_merged,cigs_perday,years_smoking,pack_years,is.family.lung,pathologyTNM
0,CRUK0034,CRUK0034,Female,68,White- Irish,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,40,Yes,No,Ex-Smoker,20.0,35,35.000,True,IB
1,CRUK0150,CRUK0150,Male,81,White- British,LCNEC,Other,NaN,Left Upper Lobe,3,0,75,Yes,No,Ex-Smoker,44.5,49,109.025,False,IIB
2,CRUK0159,CRUK0159,Male,60,White- British,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,36,Yes,Yes,Smoker,20.0,38,38.000,False,IB
3,CRUK0090,CRUK0090,Male,65,White- British,Squamous cell carcinoma,LUSC,NaN,Right Upper Lobe,2a,1,35,Yes,Yes,Ex-Smoker,10.0,35,17.500,False,IIA
4,CRUK0045,CRUK0045,Male,85,White- British,Invasive adenocarcinoma,LUAD,papillary,Left Lower Lobe,2a,0,48,Yes,Yes,Ex-Smoker,10.0,25,12.500,False,IB


In [4]:
mut_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_mutation_table.rds')
mut_table = pd.DataFrame(mut_table[None])
mut_table.head()

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False


In [5]:
clinical_df = clinical_df.sort_values(by=['cruk_id'])
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
88,CRUK0001,CRUK0001,CRUK0001,84,Male,White- British,20.0,30,30.00,Ex-Smoker,...,1120,1,1120,1,1120,0,1120,NaN,No rec,True
79,CRUK0002,CRUK0002,CRUK0002,81,Male,White- British,5.0,35,8.75,Ex-Smoker,...,1901,0,1901,0,1901,0,1901,NaN,No rec,True
26,CRUK0003,CRUK0003,CRUK0003,78,Female,White- British,0.0,0,0.00,Never Smoked,...,2304,1,217,1,217,1,217,Intra & Extra,Intra & Extra,True
23,CRUK0004,CRUK0004,CRUK0004,69,Female,White- British,10.0,27,13.50,Ex-Smoker,...,1323,1,325,1,325,1,325,Intra & Extra,Intra & Extra,True
7,CRUK0005,CRUK0005,CRUK0005,74,Female,White- British,20.0,56,56.00,Ex-Smoker,...,649,1,649,1,649,0,649,NaN,No rec,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,CRUK0861,CRUK0861,CRUK0861,65,Female,Middle eastern,10.0,45,22.50,Smoker,...,1084,0,1084,0,1084,0,1084,NaN,No rec,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.00,Ex-Smoker,...,1434,0,1434,0,1434,0,1434,NaN,No rec,False
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.00,Ex-Smoker,...,893,1,883,1,883,1,883,Intra & Extra,Intra & Extra,False
420,CRUK0881,CRUK0881_Tumour1,CRUK0881_Tumour1;CRUK0881_Tumour2,84,Male,White- British,20.0,70,70.00,Smoker,...,758,1,748,1,748,1,748,Intrathoracic,Intrathoracic,False


In [6]:
# Making the training data

y_train = []
for index, row in clinical_df.iterrows():
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train.append(1)
    else:
        y_train.append(0)

y = np.array(y_train)
print(y)

[0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1
 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1
 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0
 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 1 0 0 1 1 0]


In [7]:
gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_gene_matrix.csv')
driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_driver_matrix.csv')
fifty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_gene_matrix.csv')
fifty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_driver_matrix.csv')
twenty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_gene_matrix.csv')
twenty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_driver_matrix.csv')
ten_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/10_driver_matrix.csv')

In [13]:
gene_matrix = gene_matrix.set_index(gene_matrix.columns[0])
driver_matrix = driver_matrix.set_index(driver_matrix.columns[0])
fifty_gene_matrix = fifty_gene_matrix.set_index(fifty_gene_matrix.columns[0])
fifty_driver_matrix = fifty_driver_matrix.set_index(fifty_driver_matrix.columns[0])
twenty_gene_matrix = twenty_gene_matrix.set_index(twenty_gene_matrix.columns[0])
twenty_driver_matrix = twenty_driver_matrix.set_index(twenty_driver_matrix.columns[0])
ten_driver_matrix = ten_driver_matrix.set_index(ten_driver_matrix.columns[0])

In [9]:
mut_table.sort_values(by=['patient_id'])
mut_table

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379003,CRUK0872,CRUK0872,CRUK0872:10:120513946:C,chr10,120513946,120513946,C,T,CACUL1,exonic,...,p.S110N,0,0,659,3,S,7.0,True,subclonal,False
379004,CRUK0872,CRUK0872,CRUK0872:14:99872776:T,chr14,99872776,99872776,T,C,SETD3,intronic,...,NaN,0,0,539,3,S,7.0,True,subclonal,False
379005,CRUK0872,CRUK0872,CRUK0872:8:110131566:C,chr8,110131566,110131566,C,A,TRHR,exonic,...,p.S360X,0,0,712,3,S,7.0,True,subclonal,False
379006,CRUK0872,CRUK0872,CRUK0872:1:51032853:G,chr1,51032853,51032853,G,C,FAF1,exonic,...,p.N388K,0,0,398,3,S,9.0,True,subclonal,False


In [10]:
# Find the most common 'combTiming_SC' for each unique combination of 'patient_id' and 'Hugo_Symbol'
most_common_timing = mut_table.groupby(['patient_id', 'Hugo_Symbol'])['combTiming_SC'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else 'Other').reset_index()
most_common_timing

,patient_id,Hugo_Symbol,combTiming_SC
0,CRUK0001,A2M,subclonal
1,CRUK0001,A4GALT,subclonal
2,CRUK0001,AARS,subclonal
3,CRUK0001,AARSD1,subclonal
4,CRUK0001,AASS,subclonal
...,...,...,...
338917,CRUK0893,ZNF496,early
338918,CRUK0893,ZNF551,subclonal
338919,CRUK0893,ZNF750,early
338920,CRUK0893,ZNF831,subclonal


In [14]:
# Create a new DataFrame to store the transformed matrix
# gene_matrix = gene_matrix.set_index('patient_id')
gene_matrix_w_timing = gene_matrix.copy()

# Map values to the new gene_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the gene_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if gene_matrix_w_timing.loc[gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            gene_matrix_w_timing.loc[gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass
# Display the resulting gene_matrix_w_timing after the transformation
print("Transformed gene_matrix_w_timing:")
print(gene_matrix_w_timing)

Transformed gene_matrix_w_timing:
            ABCA13  ADAMTS12  ADAMTS16  ADAMTS20  AFF2  ANK2  APOB  ASTN1  \
patient_id                                                                  
CRUK0001         3         3         0         3     4     3     3      0   
CRUK0002         0         0         0         3     0     0     0      3   
CRUK0003         0         0         1         0     0     0     0      0   
CRUK0004         3         0         0         0     0     0     0      3   
CRUK0005         1         3         0         3     4     3     3      0   
...            ...       ...       ...       ...   ...   ...   ...    ...   
CRUK0861         0         0         0         0     4     2     2      0   
CRUK0864         0         0         0         0     0     0     0      0   
CRUK0872         0         0         0         0     0     0     1      3   
CRUK0881         0         1         3         3     0     1     1      3   
CRUK0893         0         0         0    

In [16]:
# Create a new DataFrame to store the transformed matrix
# driver_matrix = driver_matrix.set_index('patient_id')
driver_matrix_w_timing = driver_matrix.copy()

# Map values to the new driver_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the driver_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if driver_matrix_w_timing.loc[driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            driver_matrix_w_timing.loc[driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting driver_matrix_w_timing after the transformation
print("Transformed driver_matrix_w_timing:")
print(driver_matrix_w_timing)

Transformed driver_matrix_w_timing:
            ACVR1B  AMER1  APC  ARHGAP35  ARID1A  ARID1B  ARID2  ASXL1  ASXL2  \
patient_id                                                                      
CRUK0001         0      0    0         3       0       0      0      0      0   
CRUK0002         0      0    0         0       0       0      0      0      0   
CRUK0003         0      0    0         0       0       0      0      0      0   
CRUK0004         0      0    0         0       0       0      0      0      0   
CRUK0005         0      0    0         0       0       0      0      0      0   
...            ...    ...  ...       ...     ...     ...    ...    ...    ...   
CRUK0861         0      0    0         0       0       0      0      0      0   
CRUK0864         0      0    0         0       0       0      0      0      0   
CRUK0872         0      0    0         0       0       0      0      0      0   
CRUK0881         0      0    0         0       0       0      0      0   

In [17]:
# Create a new DataFrame to store the transformed matrix
# fifty_gene_matrix = fifty_gene_matrix.set_index('patient_id')
fifty_gene_matrix_w_timing = fifty_gene_matrix.copy()

# Map values to the new fifty_gene_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the fifty_gene_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if fifty_gene_matrix_w_timing.loc[fifty_gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            fifty_gene_matrix_w_timing.loc[fifty_gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting fifty_gene_matrix_w_timing after the transformation
print("Transformed fifty_gene_matrix_w_timing:")
print(fifty_gene_matrix_w_timing)


Transformed fifty_gene_matrix_w_timing:
            ABCA13  ANK2  APOB  CACNA1E  CCDC168  CNTNAP5  COL11A1  COL22A1  \
patient_id                                                                    
CRUK0001         3     3     3        0        0        3        0        3   
CRUK0002         0     0     0        0        0        0        0        0   
CRUK0003         0     0     0        0        0        0        0        0   
CRUK0004         3     0     0        0        3        0        0        0   
CRUK0005         1     3     3        3        3        0        3        3   
...            ...   ...   ...      ...      ...      ...      ...      ...   
CRUK0861         0     2     2        0        0        1        0        0   
CRUK0864         0     0     0        0        3        0        0        0   
CRUK0872         0     0     1        0        0        0        3        1   
CRUK0881         0     1     1        1        3        3        1        0   
CRUK0893    

In [18]:
# Create a new DataFrame to store the transformed matrix
# fifty_driver_matrix = fifty_driver_matrix.set_index('patient_id')
fifty_driver_matrix_w_timing = fifty_driver_matrix.copy()

# Map values to the new fifty_driver_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the fifty_driver_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if fifty_driver_matrix_w_timing.loc[fifty_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            fifty_driver_matrix_w_timing.loc[fifty_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting fifty_driver_matrix_w_timing after the transformation
print("Transformed fifty_driver_matrix_w_timing:")
print(fifty_driver_matrix_w_timing)

Transformed fifty_driver_matrix_w_timing:
            APC  ARHGAP35  ARID1A  ARID1B  ARID2  ATM  ATRX  BCLAF1  BCOR  \
patient_id                                                                  
CRUK0001      0         3       0       0      0    0     0       0     0   
CRUK0002      0         0       0       0      0    0     0       0     0   
CRUK0003      0         0       0       0      0    0     0       0     0   
CRUK0004      0         0       0       0      0    0     0       0     0   
CRUK0005      0         0       0       0      0    0     4       0     0   
...         ...       ...     ...     ...    ...  ...   ...     ...   ...   
CRUK0861      0         0       0       0      0    0     0       0     0   
CRUK0864      0         0       0       0      0    0     0       0     0   
CRUK0872      0         0       0       0      0    0     0       0     0   
CRUK0881      0         0       0       0      0    0     0       0     0   
CRUK0893      0         0       0 

In [19]:
# Create a new DataFrame to store the transformed matrix
# twenty_gene_matrix = twenty_gene_matrix.set_index('patient_id')
twenty_gene_matrix_w_timing = twenty_gene_matrix.copy()

# Map values to the new twenty_gene_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the twenty_gene_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if twenty_gene_matrix_w_timing.loc[twenty_gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            twenty_gene_matrix_w_timing.loc[twenty_gene_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting twenty_gene_matrix_w_timing after the transformation
print("Transformed twenty_gene_matrix_w_timing:")
print(twenty_gene_matrix_w_timing)

Transformed twenty_gene_matrix_w_timing:
            COL22A1  CSMD1  CSMD3  DMD  LRP1B  MUC16  MUC19  NRXN1  OBSCN  \
patient_id                                                                  
CRUK0001          3      3      3    4      0      0      3      0      3   
CRUK0002          0      0      3    0      0      3      0      0      0   
CRUK0003          0      0      3    0      1      3      0      0      3   
CRUK0004          0      0      3    0      3      3      0      0      3   
CRUK0005          3      0      3    4      1      1      0      3      3   
...             ...    ...    ...  ...    ...    ...    ...    ...    ...   
CRUK0861          0      0      1    0      0      1      0      0      0   
CRUK0864          0      0      2    4      1      2      0      0      0   
CRUK0872          1      0      1    0      1      1      1      0      0   
CRUK0881          0      3      3    4      1      1      1      3      3   
CRUK0893          0      0      0  

In [20]:
# Create a new DataFrame to store the transformed matrix
# twenty_driver_matrix = twenty_driver_matrix.set_index('patient_id')
twenty_driver_matrix_w_timing = twenty_driver_matrix.copy()

# Map values to the new twenty_driver_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the twenty_driver_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if twenty_driver_matrix_w_timing.loc[twenty_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            twenty_driver_matrix_w_timing.loc[twenty_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting twenty_driver_matrix_w_timing after the transformation
print("Transformed twenty_driver_matrix_w_timing:")
print(twenty_driver_matrix_w_timing)

Transformed twenty_driver_matrix_w_timing:
            ARID1A  ATM  ATRX  CDKN2A  COL5A2  CREBBP  EGFR  FAT1  KEAP1  \
patient_id                                                                 
CRUK0001         0    0     0       0       0       0     0     0      0   
CRUK0002         0    0     0       0       0       0     0     0      0   
CRUK0003         0    0     0       0       0       0     1     0      0   
CRUK0004         0    0     0       0       0       0     1     0      0   
CRUK0005         0    0     4       0       0       0     0     0      0   
...            ...  ...   ...     ...     ...     ...   ...   ...    ...   
CRUK0861         0    0     0       0       0       0     0     0      0   
CRUK0864         0    0     0       0       0       0     0     0      0   
CRUK0872         0    0     0       0       0       0     0     0      0   
CRUK0881         0    0     0       0       0       1     0     0      0   
CRUK0893         0    1     0       0       0

In [21]:
# Create a new DataFrame to store the transformed matrix
# ten_driver_matrix = ten_driver_matrix.set_index('patient_id')
ten_driver_matrix_w_timing = ten_driver_matrix.copy()

# Map values to the new ten_driver_matrix_w_timing
timing_mapping = {'early': 1, 'late': 2, 'subclonal': 3, 'Unknown': 4, 'NA': 4}

# transform the ten_driver_matrix_w_timing using the most_common_timing and timing_mapping
for index, row in most_common_timing.iterrows():
    try:
        if ten_driver_matrix_w_timing.loc[ten_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']].item() != 0:
            ten_driver_matrix_w_timing.loc[ten_driver_matrix_w_timing.index == row['patient_id'], row['Hugo_Symbol']] = timing_mapping[row['combTiming_SC']]
    except KeyError:
        pass

# Display the resulting ten_driver_matrix_w_timing after the transformation
print("Transformed ten_driver_matrix_w_timing:")
print(ten_driver_matrix_w_timing)

Transformed ten_driver_matrix_w_timing:
            CDKN2A  FAT1  KEAP1  KMT2D  KRAS  NF1  PIK3CA  SMARCA4  STK11  \
patient_id                                                                  
CRUK0001         0     0      0      0     0    0       0        0      0   
CRUK0002         0     0      0      0     0    3       0        0      0   
CRUK0003         0     0      0      0     0    0       2        0      0   
CRUK0004         0     0      0      0     0    0       0        0      0   
CRUK0005         0     0      0      0     0    0       0        0      0   
...            ...   ...    ...    ...   ...  ...     ...      ...    ...   
CRUK0861         0     0      0      0     0    0       0        0      0   
CRUK0864         0     0      0      0     1    0       0        0      0   
CRUK0872         0     0      0      0     0    0       0        0      0   
CRUK0881         0     0      0      0     1    1       0        0      0   
CRUK0893         0     0      0     

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Best parameters:  {'C': 1}
Best score:  0.5498217468805704
Predictions:  [0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1
 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1
 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5102600250626566
ROC AUC:  0.4699248120300752
               precision    recall  f1-score   support

No Recurrence       0.67      0.67      0.67       112
   Recurrence       0.35      0.35      0.35       

In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (15,)}
Best score:  0.5680926916221034
Predictions:  [1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0
 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 0 1
 1 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0
 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5281171679197996
ROC AUC:  0.5012531328320802
               precision    recall  f1-score   support

No Recurrence       0.68      0.71      0.69       112
   Recurrence  

In [24]:
X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)
Best parameters:  {'C': 0.1}
Best score:  0.5393939393939393
Predictions:  [0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0
 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.46749686716791977
ROC AUC:  0.43217418546365916
               precision    recall  f1-score   support

No Recurrence       0.64      0.79      0.71       112
   Recurrence

In [25]:
X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (10,)}
Best score:  0.5591800356506239
Predictions:  [0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0
 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1
 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4396146616541353
ROC AUC:  0.4231672932330827
               precision    recall  f1-score   support

No Recurrence       0.62      0.62      0.62       112
   Recurrence  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)
Best parameters:  {'C': 10}
Best score:  0.5158645276292335
Predictions:  [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1
 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.46749686716791977
ROC AUC:  0.44689849624060146
               precision    recall  f1-score   support

No Recurrence       0.64      0.79      0.71       112
   Recurrence 

In [27]:
X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (10,)}
Best score:  0.5590017825311943
Predictions:  [0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0
 0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 0
 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0
 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5056390977443609
ROC AUC:  0.5153508771929824
               precision    recall  f1-score   support

No Recurrence       0.67      0.64      0.65       112
   Recurrence  

In [28]:
X_train, X_test, y_train, y_test = train_test_split(driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Best parameters:  {'C': 1}
Best score:  0.5003565062388592
Predictions:  [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4714912280701754
ROC AUC:  0.4634241854636592
               precision    recall  f1-score   support

No Recurrence       0.65      0.75      0.69       112
   Recurrence       0.28      0.19      0.23       

In [29]:
X_train, X_test, y_train, y_test = train_test_split(driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (5,)}
Best score:  0.5535650623885919
Predictions:  [1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1
 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1
 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 0 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1
 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4656171679197995
ROC AUC:  0.43060776942355894
               precision    recall  f1-score   support

No Recurrence       0.64      0.58      0.61       112
   Recurrence   

In [30]:
X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)
Best parameters:  {'C': 10}
Best score:  0.5241532976827095
Predictions:  [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4537907268170426
ROC AUC:  0.4797149122807018
               precision    recall  f1-score   support

No Recurrence       0.64      0.73      0.68       112
   Recurrence   

In [31]:
X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 0.001, 'hidden_layer_sizes': (1,)}
Best score:  0.5488413547237078
Predictions:  [1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5078320802005013
ROC AUC:  0.5026629072681704
               precision    recall  f1-score   support

No Recurrence       0.67      0.89      0.76       112
   Recurrence  

In [32]:
X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)
Best parameters:  {'C': 100}
Best score:  0.5426916221033868
Predictions:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.49906015037593987
ROC AUC:  0.5089285714285714
               precision    recall  f1-score   support

No Recurrence       0.66      0.89      0.76       112
   Recurrence 

In [33]:
X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (15,)}
Best score:  0.5946524064171123
Predictions:  [0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0
 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0
 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 1
 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5148809523809523
ROC AUC:  0.537593984962406
               precision    recall  f1-score   support

No Recurrence       0.67      0.70      0.68       112
   Recurrence    

In [34]:
X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)
Best parameters:  {'C': 10}
Best score:  0.5464349376114083
Predictions:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.48151629072681706
ROC AUC:  0.47564223057644106
               precision    recall  f1-score   support

No Recurrence       0.65      0.89      0.75       112
   Recurrence 

In [35]:
X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix_w_timing, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Best parameters:  {'alpha': 1, 'hidden_layer_sizes': (15,)}
Best score:  0.5721925133689839
Predictions:  [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4631892230576441
ROC AUC:  0.443687343358396
               precision    recall  f1-score   support

No Recurrence       0.64      0.80      0.71       112
   Recurrence      

In [79]:
# code to make the cumulative matrix - 100-10 Driver/Non-driver, LUAD/LUSC, Ploidy, Timing
# only with 394 patients (ploidy = NA are removed)

LUAD_patient_id = []
LUSC_patient_id = []

for index, row in tumor_df.iterrows():
    if row['histology_3'] == 'LUAD':
        LUAD_patient_id.append(row['patient_id'])
    elif row['histology_3'] == 'LUSC':
        LUSC_patient_id.append(row['patient_id'])

cumulative_gene_matrix = gene_matrix_w_timing.copy()
cumulative_gene_matrix['LUAD']= cumulative_gene_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_gene_matrix['LUSC']= cumulative_gene_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_driver_matrix = driver_matrix_w_timing.copy()
cumulative_driver_matrix['LUAD']= cumulative_driver_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_driver_matrix['LUSC']= cumulative_driver_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_fifty_gene_matrix = fifty_gene_matrix_w_timing.copy()
cumulative_fifty_gene_matrix['LUAD']= cumulative_fifty_gene_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_fifty_gene_matrix['LUSC']= cumulative_fifty_gene_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_fifty_driver_matrix = fifty_driver_matrix_w_timing.copy()
cumulative_fifty_driver_matrix['LUAD']= cumulative_fifty_driver_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_fifty_driver_matrix['LUSC']= cumulative_fifty_driver_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_twenty_gene_matrix = twenty_gene_matrix_w_timing.copy()
cumulative_twenty_gene_matrix['LUAD']= cumulative_twenty_gene_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_twenty_gene_matrix['LUSC']= cumulative_twenty_gene_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_twenty_driver_matrix = twenty_driver_matrix_w_timing.copy()
cumulative_twenty_driver_matrix['LUAD']= cumulative_twenty_driver_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_twenty_driver_matrix['LUSC']= cumulative_twenty_driver_matrix.index.isin(LUSC_patient_id).astype(int)

cumulative_ten_driver_matrix = ten_driver_matrix_w_timing.copy()
cumulative_ten_driver_matrix['LUAD']= cumulative_ten_driver_matrix.index.isin(LUAD_patient_id).astype(int)
cumulative_ten_driver_matrix['LUSC']= cumulative_ten_driver_matrix.index.isin(LUSC_patient_id).astype(int)

In [85]:
cna_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_scna_table.rds')
# Obtain dataframe and rename column sample to RegionID
cna_table = pd.DataFrame(cna_table[None])
cna_table_no_duplicates = cna_table.drop_duplicates(subset='patient_id')
cna_table_no_duplicates = cna_table_no_duplicates.sort_values(by=['patient_id'])

In [86]:
cna_table_no_duplicates

,patient_id,tumour_id,RegionID,chr,startpos,endpos,n.het,cnTotal,nMajor,nMinor,Ploidy,ACF,nAraw,nBraw,cpn_event_vs_ploidy,MSAI,samp_used_to_phase
52958,CRUK0001,CRUK0001,CRUK0001:R1,1,762273,3800149,134,4,2,2,4.493770,0.22,2.073981,1.625892,neutral,True,CRUK0001_SU_T1-R3
288283,CRUK0002,CRUK0002,CRUK0002:R1,1,762273,2537166,72,3,2,1,1.921907,0.38,1.696286,1.044553,gain,False,CRUK0002_SU_T1-R3
141069,CRUK0003,CRUK0003,CRUK0003:R1,1,752894,3427608,110,3,2,1,3.567392,0.54,2.041043,1.081540,neutral,False,CRUK0003_SU_T1-R2
45121,CRUK0004,CRUK0004,CRUK0004:R1,1,762496,3496497,53,3,2,1,2.950603,0.23,1.804428,1.409574,neutral,False,CRUK0004_SU_T1-R3
0,CRUK0005,CRUK0005,CRUK0005:R1,1,762273,2252860,73,3,2,1,3.131282,0.15,2.463756,1.184761,neutral,False,CRUK0005_SU_T1-R2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368701,CRUK0861,CRUK0861,CRUK0861:R1,1,752894,1572735,79,4,4,0,3.514533,0.76,4.050553,0.000000,neutral,False,CRUK0861_SU_T1-R2
375529,CRUK0864,CRUK0864,CRUK0864:R1,1,762273,1448024,90,3,2,1,3.556913,0.31,2.074464,1.035148,neutral,False,CRUK0864_SU_T1-R2
396701,CRUK0872,CRUK0872,CRUK0872:R1,1,762273,990280,34,4,2,2,3.919491,0.26,2.102410,1.801966,neutral,False,CRUK0872_SU_T1-R2
417335,CRUK0881,CRUK0881_Tumour1,:R1,1,65797,1027553,39,3,2,1,4.186610,0.33,1.992436,0.923056,neutral,False,CRUK0881_SU_T1-R5


In [91]:
cumulative_gene_matrix['Ploidy'] = cumulative_gene_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_gene_matrix = cumulative_gene_matrix.dropna(subset=['Ploidy'])
cumulative_driver_matrix['Ploidy'] = cumulative_driver_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_driver_matrix = cumulative_driver_matrix.dropna(subset=['Ploidy'])
cumulative_fifty_gene_matrix['Ploidy'] = cumulative_fifty_gene_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_fifty_gene_matrix = cumulative_fifty_gene_matrix.dropna(subset=['Ploidy'])
cumulative_fifty_driver_matrix['Ploidy'] = cumulative_fifty_driver_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_fifty_driver_matrix = cumulative_fifty_driver_matrix.dropna(subset=['Ploidy'])
cumulative_twenty_gene_matrix['Ploidy'] = cumulative_twenty_gene_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_twenty_gene_matrix = cumulative_twenty_gene_matrix.dropna(subset=['Ploidy'])
cumulative_twenty_driver_matrix['Ploidy'] = cumulative_twenty_driver_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_twenty_driver_matrix = cumulative_twenty_driver_matrix.dropna(subset=['Ploidy'])
cumulative_ten_driver_matrix['Ploidy'] = cumulative_ten_driver_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])
cumulative_ten_driver_matrix = cumulative_ten_driver_matrix.dropna(subset=['Ploidy'])

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_75587/1003316725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_gene_matrix['Ploidy'] = cumulative_gene_matrix.index.map(cna_table_no_duplicates.set_index('patient_id')['Ploidy'])


In [95]:
cumulative_gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_100_gene_matrix.csv')
cumulative_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_100_driver_matrix.csv')
cumulative_fifty_gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_50_gene_matrix.csv')
cumulative_fifty_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_50_driver_matrix.csv')
cumulative_twenty_gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_20_gene_matrix.csv')
cumulative_twenty_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_20_driver_matrix.csv')
cumulative_ten_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/cumulative_10_driver_matrix.csv')

In [8]:
#calculate the sum of all numbers in y 
sum = 0
for i in y:
    sum += i
print(sum)

142
